# **Sentiment Analysis Services Using an AI Chatbot**: Helping Users Analysing tweets and Browse Products on Amazon.in Using Chatbot




##**INTRODUCTION**


As part of the Final Project for AI3 C4 Univ.ai's Course, the team-  Ratna Sambhav, Suprit Choudhary and Jainam Doshi, had working on developing a chatbot which can solve user's problem related to getting suitable product on Amazon.in and understanding sentiment of tweets on Twitter.<br>
<br>
**As we all know, We need people's advice and opinions on every stage of life!**
<br>
<br>

We always rely on the opinions of other people for making any decision. Even when we buy a product online, we check the reviews of other customers who bought the product. When an event occurs or is taking place, we want to know how the world or people of any particular area thinks about the ongoing event and based on that we form our own opinions about the event. But manually going through all the news and product reviews can be very time taking if not frustrating. So our goal is to make a platform for sentiment analysis on Amazon product reviews, Twitter Tweets about an event and also comments of people on Reddit regarding any product or news. Integrating these three services will be an AI chatbot which will use sentence embeddings to understand the user’s intentions and give him the desired services accordingly. 

<br>
<br>

We all face the need for sentiment analysis but there can’t be anything better than using an interactive chatbot to get the job done.Most of the chatbots used by companies are mainly rule based. Also, the method to develop an AI chatbot requires the use of paid services from Watson Assistant, AWS Lex chatbot and the companies are willing to pay for these high priced services. But we already have pretty impressive models trained on huge datasets available for free on HuggingFace which can be easily integrated to make intelligent chatbots. Through this project, we are trying to prove this point and also (if we get able to launch an app on it) have an app whether people can go, talk to the chatbot and get their work done.
<br>
<br>
**Important Note: Twitter Sentiment Analysis Models training are in separate notebook in the same folder. For keeping the notebook clean and comprehensible, this was done.**



## Ground Work

In [198]:
!pip install emoji==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [199]:
# Updating the application repository and installing chrome driver
!apt update 
!apt install chromium-chromedriver 
!pip install -q selenium webdriver_manager 

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 264 kB in 2s (116 kB/s)
Reading p

In [200]:
# Setup 
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

#options ------------------ 
options = webdriver.ChromeOptions()

options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')



#NOTE :  if you download the notebook on your local machine,comment out this 
# option. You will be able to see the browser GUI in action.
options.add_argument('--headless')  

In [201]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [202]:
## Open and close a browser with selenium
#start the browser
driver = webdriver.Chrome(options=options)

#get the current title
driver.title

''

In [203]:
## IMPORTING AND INSTALLING NECESSARY LIBRAIRES

!pip install configparser
import requests
import re
import os
import zipfile
import string
import collections
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
import urllib.request
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline
from IPython.core.display import HTML
import nltk
nltk.download('punkt')
from nltk import word_tokenize,sent_tokenize
from keras.layers import  Bidirectional
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Embedding, Dense
from collections import defaultdict

from sklearn.model_selection import train_test_split
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import matplotlib.pyplot as plt
from progressbar import ProgressBar
import spacy
sp = spacy.load('en_core_web_sm')
!pip install tweepy
!pip install snscrape
import snscrape.modules.twitter as sntwitter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Amazon Reviews Sentiment Analysis**

##### Just the HuggingFace Model :)

In [ ]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model='finiteautomata/bertweet-base-sentiment-analysis')

# Function
def amazon_review_sentiment(list_review):
  
  pos_count = 0 # positive count
  neg_count = 0 # negative count

  # Using foor loop we check the statement is positive or negative
  for i in list_review:
    result = classifier(i)
    if (result[0]['label'] == 'NEU'):
      neg_count+=1
    elif (result[0]['label'] == 'POS'):
      pos_count+=1

  # Calculte the percentage of positive and negative reviews in the list
  positive_percentage = (pos_count*100)/len(list_review)
  negative_percentage = (neg_count*100)/len(list_review)

  print(f'positive review is {positive_percentage}% and negative review is {negative_percentage}%')

## Amazon on demand Scraping: 
Amazon product and review scraping of user choice

#### Python Function
##### INPUT ---->  Url of the amazon products list page <BR>
##### OUTPUT ------>  All the product names, their prices, their image link and the links to all their individual pages

In [ ]:
def get_product_data(url):
  while True:
    driver.get(url = url)
    name, price, image, link = [], [], [], []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    list_of_product = soup.find_all("span", {"data-component-type": "s-search-results"})
    for i in range(1, 40):
      try:
        try:
          name.append(list_of_product[0].find('div', {"data-index": str(i)}).find('span', {'class' : 'a-size-medium a-color-base a-text-normal'}).text)
        except:
          name.append(list_of_product[0].find('div', {"data-index": str(i)}).find('span', {'class' : 'a-size-base-plus a-color-base a-text-normal'}).text)
        price.append(list_of_product[0].find('div', {"data-index": str(i)}).find('span', {'class' : 'a-price-whole'}).text)
        link.append('https://www.amazon.in' + list_of_product[0].find('div', {"data-index": str(i)}).find('span', {'data-component-type' : 's-product-image'}).a['href'])
        image.append(list_of_product[0].find('div', {"data-index": str(i)}).find('span', {'data-component-type' : 's-product-image'}).div.img['src'])
      except:
        continue

    if len(name) != 0:
      break
  return (name, price, image, link)

# get_product_data('https://www.amazon.in/s?k=smart+phones&crid=2N9O04J6K0HGX&sprefix=smart+phones%2Caps%2C322&ref=nb_sb_noss_1')

#### Python Function
##### INPUT ---->  Url of the current amazon products list page <BR>
##### OUTPUT ------>  Url of next products list page

In [ ]:
def link_to_next_product_page(url):
  while True:
    try:
      driver.get(url = url)
      soup = BeautifulSoup(driver.page_source, 'html.parser')
      pagination = soup.find_all("span", {"class": "s-pagination-strip"})[0]
      break
    except:
      continue
  next_page = pagination.find_all('a', {"class":"s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"})[0]['href']
  # print(next_page)
  return "https://www.amazon.in" + next_page

#### Python Function
##### INPUT ---->  Name of the product (to search for) <BR>
##### OUTPUT ------>  Information of all the products that appear after an amazon search (all the pages)

In [ ]:
def get_all_products(product_name):
  result_list = []
  while True:
    try:
      # open the website 
      driver.get(url = 'https://www.amazon.in')

      # Find the search box
      header1_tags = driver.find_elements(By.ID, 'twotabsearchtextbox')
      search_box = header1_tags[0]

      # Send the key command and get the url of the new page
      search_box.clear()
      search_box.send_keys(product_name  +  Keys.ENTER)

      # Provide the page to beautiful soup
      page_url = driver.current_url
      # Use the function to get all the products in the page
      results = get_product_data(page_url)
      result_list.append(results)
      break
    except:
      continue

  # Get next page link: If next page link exists
  # The following code will run infinitely going to the next page and collecting product info and if next page link is not found it will break
  while True:
    try:
      # Try to get the link of next page: If it fails, then that will mean there is no next page and break will occur in exception
      next_product_page = link_to_next_product_page(driver.current_url)
      results = get_product_data(next_product_page)
    except:
      # print("didn't work")
      break
    finally:
      result_list.append(results)

  return result_list

#### Python Function
##### INPUT ---->  A list containing links of images <BR>
##### OUTPUT ------>  List containing images

In [ ]:
import requests
import shutil
from PIL import Image, ImageDraw, ImageFont, ImageOps

def display_imgs_from_link_array(url_array):
  image_list = []
  for i,url in enumerate(url_array):
    path = '/content/myIMG' + str(i) + '.jpg'
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(path, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)  
    image = Image.open(path)
    image_list.append(image)
  return image_list

# im = display_imgs_from_link_array(product_r[0][2])

#### Python Function
##### INPUT ---->  A very long sentence <BR>
##### OUTPUT ------>  Sentences of appropriate sizes separated by line break

In [ ]:
def split_sentence(sentence):
  fg = sentence.split(' ')
  fg_l = [' '.join(fg[i:i+5]) for i in range(0, len(fg), 5)]
  fg_l = '\n'.join(fg_l)
  return fg_l

# print(split_sentence(product_r[0][0][5]))

#### Python Function
##### INPUT ---->  List containing images, another lists containing product names and product prices. <BR>
##### OUTPUT ------>  A single large size image containing in an organized way, all the product info and images.

In [ ]:
def display_all_product(product_name, product_price, image_listing):
  text_box_width = 400
  jombo_img_width = text_box_width + 10 + max([i.size[0] for i in image_listing])
  jombo_img = Image.new("RGB", (jombo_img_width, sum([i.size[1]+50 for i in image_listing])), (255, 255, 255))
  y_size = 0
  for i,im in enumerate(image_listing):
    # Add Text to an image
    text = Image.new("RGB", (text_box_width, im.size[1]), (255, 255, 255))
    I1 = ImageDraw.Draw(text)
    myfont = ImageFont.truetype(r'/content/160MKA__.TTF', 20)
    myfont2 = ImageFont.truetype(r'/content/160MKA__.TTF', 30)
    I1.text((10, 10), split_sentence(product_name[i]), font=myfont, fill=(0, 0, 0))
    I1.text((10, im.size[1]-100), 'Rs.'+ product_price[i], font=myfont, fill=(255, 0, 255))
    I1.text((10, im.size[1]-50), str(i), font=myfont2, fill=(0, 0, 0))
    border_text = (5, 0, 0, 0)
    new_text = ImageOps.expand(text, border=border_text, fill='green')

    jombo_img.paste(im, (jombo_img_width-im.size[0], y_size))
    jombo_img.paste(new_text, (0, y_size))
    y_size += im.size[1]+50
  return jombo_img

# display_all_product(product_name, product_price, image_listing)

#### Python Function
##### INPUT ---->  Keywork for searching amazon.in and other two variables <BR>
##### OUTPUT ------>  Displays all the products and their images in an organized way and also returns it

In [ ]:
def show_the_products(keyword, page_number=0, same_product=0):

  # Access the products' info
  if same_product == 0:
    global product_r
    product_r = get_all_products(keyword)
    product_name = product_r[page_number][0]
    product_price = product_r[page_number][1]
    product_image_link = product_r[page_number][2]
    product_link = product_r[page_number][3]
  elif same_product != 0:
    product_name = product_r[page_number][0]
    product_price = product_r[page_number][1]
    product_image_link = product_r[page_number][2]
    product_link = product_r[page_number][3]

  # Get images from the array of links
  image_listing = display_imgs_from_link_array(product_image_link)

  # Get a Jumbo image (PIL format) that contains all the images and product name and price
  all_products = display_all_product(product_name, product_price, image_listing)

  return all_products

### Extract Reviews

#### Python Function
##### INPUT ---->  Url of the amazon product choosen by user <BR>
##### OUTPUT ------>  Url of the review page

In [ ]:
def get_all_reviews_page_link(product_page_url):
  # Try to get the current page which is an amazon page
  n = 0
  while n<5:
    try:
      driver.get(url = product_page_url)
      driver.find_elements(By.ID, 'productTitle')
      soup = BeautifulSoup(driver.page_source, 'html.parser')
      link = soup.find('a', {'data-hook' : 'see-all-reviews-link-foot'})['href']
      return 'https://www.amazon.in' + link
      # break
    except:
      n += 1

# get_all_reviews_page_link('https://www.amazon.in/ALUMINIUM-EXTRUSION-OPENBUILDS-PROFILE-PRINTERS/dp/B076BJ1S1S/ref=sr_1_21_sspa?crid=15NVZMO3D7N8D&keywords=3d+printer&qid=1670892864&sprefix=3d+printer%2Caps%2C247&sr=8-21-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9tdGY&psc=1&smid=A3QR25AL37YGZY')

#### Python Function
##### INPUT ---->  Url of any review page of an amazon product <BR>
##### OUTPUT ------>  Url of the next review page

In [ ]:
def get_next_review_page_link(current_review_page):
  n = 0
  while n < 5:
    try:
      driver.get(url = current_review_page)
      driver.find_elements(By.CLASS_NAME, 'a-profile')
      pagination = driver.find_elements(By.CLASS_NAME, 'a-pagination')[0]
      # print('Pagination found')
      next_link = pagination.find_elements(By.TAG_NAME, 'li')[-1].find_elements(By.TAG_NAME, 'a')[0].get_attribute('href')
      # print('link found')
      return next_link
    except:
      n += 1

# get_next_review_page_link('https://www.amazon.in/Official-Creality-Motherboard-Carborundum-220x220x250mm/product-reviews/B08D65ZH5W/ref=cm_cr_getr_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2')

#### Python Function
##### INPUT ---->  Url of any review page of any amazon product <BR>
##### OUTPUT ------>  All the reviews on that page

In [ ]:
def get_all_reviews_of_page(review_page):
  cleaned_reviews = []
  while len(cleaned_reviews) == 0:
    try:
      driver.get(url = review_page)
      driver.find_elements(By.CLASS_NAME, 'a-profile')
      soup = BeautifulSoup(driver.page_source, 'html.parser')
      all_reviews = soup.find_all('span', {'data-hook' : 'review-body'})
      cleaned_reviews = [i.text.replace('\n', '') for i in all_reviews]
    except:
      continue
  return cleaned_reviews

# get_all_reviews_of_page('https://www.amazon.in/Official-Creality-Motherboard-Carborundum-220x220x250mm/product-reviews/B08D65ZH5W/ref=cm_cr_getr_d_paging_btm_prev_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2')

#### Python Function
##### INPUT ---->  Url of the amazon products chosen by the user <BR>
##### OUTPUT ------> A list of all the reviews given for that product

In [ ]:
def get_reviews_for_product(product_page):
  review_list = []
  review_pages = get_all_reviews_page_link(product_page)
  if review_pages:
    review_list += get_all_reviews_of_page(review_pages)
    while True:
      next_review_page = get_next_review_page_link(review_pages)
      if next_review_page:
        review_list += get_all_reviews_of_page(next_review_page)
        review_pages = next_review_page
      else:
        break
    return review_list
  else:
    print('There are no reviews for this product.')
    return None

# get_reviews_for_product('https://www.amazon.in/gp/product/B0BGPKCGGG/ref=s9_acss_bw_cg_Header_2a1_w?pf_rd_m=A1K21FY43GMZF8&pf_rd_s=merchandised-search-5&pf_rd_r=22PFBNZ9BWHWWS09SX11&pf_rd_t=101&pf_rd_p=1d01e7f9-f97a-4c5e-b534-577595d3450a&pf_rd_i=1380365031')

## **Chatbot**: A Basic WorkFlow of Intention, Dialogue and Action.

##### <b> Emotion Analysis Model </b> : Loading Distil-Roberta Base model from HuggingFace for emotion analysis in the greeting.

In [ ]:
!pip install transformers
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 44.6 MB/s 
     |████████████████████████████████| 182 kB 70.5 MB/s 


Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/329M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/294 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


##### <b> Question-Answering Model </b> : Loading Roberta model fine tuned on Squad2 dataset for Question-answering

In [ ]:
model_name = "deepset/roberta-base-squad2"
qa_model = pipeline('question-answering', model=model_name, tokenizer=model_name)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

##### <b> Sentence EmbeddingModel </b> : Loading MiniLM model for sentence embedding

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def cosine(x, y):
  return np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 1.3 MB 36.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f563c6b93d1119f80037aef40372ca0cba9f11f9973020ce58068ec15f3cc7ba
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

##### Loads font, necessary libraries for chatbot and function to display chatbot responses.

In [ ]:
!wget 'https://github.com/Ratna-Sambhav/myfontdata/raw/main/160MKA__.TTF'
myfont = ImageFont.truetype(r'/content/160MKA__.TTF', 20)

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display, HTML

def mprint(mystr):
  display(HTML(f'<font size="5" face="comic sans" color={np.random.choice(["blue", "green", "red", "brown"])}>  >> {mystr}   </font>'))

--2022-12-15 08:35:35--  https://github.com/Ratna-Sambhav/myfontdata/raw/main/160MKA__.TTF
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Ratna-Sambhav/myfontdata/main/160MKA__.TTF [following]
--2022-12-15 08:35:35--  https://raw.githubusercontent.com/Ratna-Sambhav/myfontdata/main/160MKA__.TTF
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27816 (27K) [application/octet-stream]
Saving to: ‘160MKA__.TTF’

160MKA__.TTF        100%[===================>]  27.16K  --.-KB/s    in 0.001s  

2022-12-15 08:35:35 (17.7 MB/s) - ‘160MKA__.TTF’ saved [27816/27816]



#### **Chatbot Main Functions**

##### **Greetings**:
1. Greet the User.
2. Detects user emotion from his response: Using the emotion model
3. Prints a dialogue based on user's emotion from a given list of sentences.

In [ ]:
def greetings():
  mprint('Hi, How are you?')
  classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")
  user_input = input()
  reply = {'anger': "Don't worry much. Have a look on the services I provide. Would you?\n",\
           'disgust' : "Oh I understand. I can provide you some information that you might need. Wanna know what services I provide\n",\
           'fear': "Oh ok. But don't worry about it much. I can provide you some information that you might need. Wanna know what services I provide\n", \
           'joy': "Wow Nice to know. Can I showcase the services that I provide?\n", \
           'neutral': "Can I showcase the services that I provide?\n", \
           'sadness': "I am truly sorry about that. Lets move forward and ask the capabilities of this chatbot.\n",\
           'surprise': "Wow, it's great. Now can I show the services I provide?\n"}
  clas = classifier(user_input)
  if clas[0]['score'] < 0.5:
    mprint(reply['neutral'])
  else:
    mprint(f"I can say with {round(100*clas[0]['score'], 2)}% confidence that your current emotional state is {clas[0]['label']}. " + reply[clas[0]['label']])
  mprint("(Or if you already know about this Chatbot, Tell directly which service you would like)\n")

##### **About Chatbot Services**:
1. Ask the user.
2. Use sentence embedding model to detect whether he wants to know details about the service or go to directly the given service.
3. Show the user details of the services or go to the desired services directly (amazon or twitter) based on user's response.

In [ ]:
def about_service():
  user_input = input()
  input_embedding = embedding_model.encode([user_input])
  service_corpus = {'services':['Yes', 'Ok', 'show me services you provide', 'I would like to know your capability then', 'Yeah sure', 'Fine'],\
            'amazon': ['I would like to use the Amazon service', 'Open the amazon service directly', 'I want to browse the amazon products', 'Your second service'], \
            'twitter': ['I would like to use the Twitter service', 'Open the Twitter service directly', 'I want to twitter sentiment analysis', 'Your first service', 'I would like to see what people say about this thing on twitter']}
  
  # Checking with which type of sentencem user's sentence is having max cosine similarity
  user_intention_list = [np.max([cosine(input_embedding, i) for i in embeddings]) for embeddings in [embedding_model.encode(types) for types in service_corpus.values()]]
  user_intention = list(service_corpus.keys())[np.argmax(user_intention_list)]
  if user_intention == 'services':
    mprint("Here are the details: \n\t1. Twitter Sentiment Analysis: Our custom build model provides accurate sentiment prediction. \
    User gives a topic and the chatbot will tell you whether people tweeting about this topic on Twitter are positive or negative about the topic.\
    \n\t2. Amazon Product Analysis: Tell us what you want to buy from Amazon.in and the chatbot will let you browse the products and give whether people\
    are happy or not happy about the product in percentage\n\n")
    know_more_about_or_get_to_service()
  if user_intention == 'twitter':
    twitter_service()
  if user_intention == 'amazon':
    amazon_service()


In [ ]:
Twitter_Detail = "We built three models for sentiment analysis compred them and chose the one with max accuracy. Following are the models we built:\
\nCustom model - We will be using glove/word2vec encoding with a custom model of Lstm and dropout layers \
\nGpt- We are trying to use a pre-trained GPT model with GPT tokenizer and fine tune it for our data. \
\nBert-We will be using BERT architecture for single sentence classification, specifically the architecture used for CoLA (Corpus of Linguistic Acceptability) binary classification task. \
\nWe are also trying a few other models depending on time. \
\nFinally we'll extract tweets based on keywords  imputed by the user using snscrape  and use those tweets to predict.\
\n\n We shall be using the model with the best accuracy for our final chatbot. Dataset: https://www.kaggle.com/datasets/kazanova/sentiment140?resource=download \n"

Amazon_detail = "We use the amazon review dataset. First, we extract the product features as keywords using the spaCy library. Then we will do sentiment analysis based on keywords that are present in the reviews so we got the analysis of the features are good or bad and the overall user experience based on that feature. We use the pre-build model from the hugging face for sentiment analysis. \
\nWe will make ChatBot for the user who selects the category and give the name of the product so the bot scrapes the data from amazon and get the analysis for the product and its features. \
\nThe dataset we use is mentioned below which is categories. So it helps us to extract the keywords for the products. \
\nDataset: https://jmcauley.ucsd.edu/data/amazon_v2/index.html \n"








# TO KNOW MORE ABOUT THE SERVICE
def know_more_about_or_get_to_service():
  mprint("Now choose one of the service. If you want to more details, say so.")
  user_input = input()
  input_embedding = embedding_model.encode([user_input])
  service_corpus = {'services':['I want to more about the services in detail', 'Tell me about the models you are using', 'Tell me how are you accessing the API', \
                                'I want to know more', 'Yes tell me more'],\
            'amazon': ['I would like to use the Amazon service', 'Open the amazon service', 'I want to browse the amazon products', 'Your second service'], \
            'twitter': ['I would like to use the Twitter service', 'Open the Twitter service', 'I want to twitter sentiment analysis', 'Your first service', 'I would like to see what people say about this thing on twitter']}
  
  # Checking with which type of sentencem user's sentence is having max cosine similarity
  user_intention_list = [np.max([cosine(input_embedding, i) for i in embeddings]) for embeddings in [embedding_model.encode(types) for types in service_corpus.values()]]
  user_intention = list(service_corpus.keys())[np.argmax(user_intention_list)]
  if user_intention == 'services':
    mprint(f"Here are the details: \n\t1. Twitter Sentiment Analysis: {Twitter_Detail}\
                    \n\t2. Amazon Product Analysis: {Amazon_detail}\n")
    use_service_or_end()
  if user_intention == 'twitter':
    twitter_service()
  if user_intention == 'amazon':
    amazon_service()

# TO GIVE MORE DETAIL ABOUT THE SERVICE AND THEN END THE CONVERSATION OR GO TOO A SERVICE
def use_service_or_end():
  mprint("It's the end of all information about this chatbot. Now tell me which service you want to use or would you like to end here\n")
  user_input = input()
  input_embedding = embedding_model.encode([user_input])
  service_corpus = {'services':['No just end it here', 'I dont want to know anymore', 'No, I dont want to use any of the service'],\
            'amazon': ['I would like to use the Amazon service', 'Open the amazon service', 'I want to browse the amazon products', 'Your second service'], \
            'twitter': ['I would like to use the Twitter service', 'Open the Twitter service', 'I want to twitter sentiment analysis', 'Your first service', 'I would like to see what people say about this thing on twitter']}
  
  # Checking with which type of sentencem user's sentence is having max cosine similarity
  user_intention_list = [np.max([cosine(input_embedding, i) for i in embeddings]) for embeddings in [embedding_model.encode(types) for types in service_corpus.values()]]
  user_intention = list(service_corpus.keys())[np.argmax(user_intention_list)]
  if user_intention == 'services':
    mprint("Thankyou for your time.")
  if user_intention == 'twitter':
    twitter_service()
  if user_intention == 'amazon':
    amazon_service()

#### **Amazon Service**

In [ ]:
def amazon_service():
  mprint("####  LET'S BEGIN AMAZON #####")
  mprint("\nJust think about what would you like to browse on amazon.in. Any cool gadget, smart phones, anything..")
  amazon_service1()
  amazon_service2()
  amazon_service3()
  mprint('ThankYou for your time.')

##### **Function 1 of Amazon Service**: 
1. Collects the product data from amazon based on a user given keyword.
2. Also can go to next page of a=the aleady collected product data
3. Asks the user to select a product from the list or repeat again.

In [ ]:
import time
def amazon_service1(page=0, again=0):
  if again!=0:
    mprint('Type again what you want to browse')
    user_input = input()
    mprint('\nOk, now again give me some time for fetching the data.\n')
    all_product_image = show_the_products(keyword=user_input, page_number=page, same_product=0)
  elif page==0 and again==0:
    global current_page
    current_page = 0
    user_input = input()
    mprint('\nOk, now give me some time for fetching the data.\n')
    all_product_image = show_the_products(keyword=user_input, page_number=page, same_product=0)
  elif page!=0:
    current_page += 1
    mprint("\nOk, This time I won't take much time.\n")
    all_product_image = show_the_products(keyword=None, page_number=current_page, same_product=1)
  saved = all_product_image.save('/content/all_product_image.jpg')
  all_product_image = Image.open('/content/all_product_image.jpg')
  display(all_product_image)
  time.sleep(2)
  mprint("\nHave you seen the products....  Now choose one (Only Integers Please). \nOr tell me search again, I'll do that. \nI can also browse more.")

##### **Function 2 of Amazon Service**: 
1. Asks the user to input the product number of the product which he chose.
2. Repeat step 1 if user types his choice in a form not integer.
3. Search again if the user wants to or go to next page.

In [ ]:
def amazon_service2():
  while True:
    user_input = input()
    try: 
      number = int(user_input)
      mprint(f'Number {number} is an excellent choice')
      mprint('I have the capability of analysing these reviews.\nWould you like to ask questions about the product (which will be answered from what reviewers\
      are telling about the product. \nOr I can give you the percentage of happy and not happy customers for this product.)')
      global user_product_number_choice
      user_product_number_choice = number
      break
    except:
      input_embedding = embedding_model.encode([user_input])
      service_corpus = {'numbers':['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seven', 'twenty first', 'thirty second', 'eleventh', 'twelfth', 'nineteenth'],\
                'browse_more': ['Show me more', 'Load more', 'More products', 'Next page', 'Keep loading', 'second page'], \
                'search_again': ['Please search again', 'start over', 'Begin again', 'Another product', 'I want to browse other product', 'not this product']}
      
      # Checking with which type of sentencem user's sentence is having max cosine similarity
      user_intention_list = [np.max([cosine(input_embedding, i) for i in embeddings]) for embeddings in [embedding_model.encode(types) for types in service_corpus.values()]]
      user_intention = list(service_corpus.keys())[np.argmax(user_intention_list)]
      if user_intention == 'numbers':
        mprint("Please type in integer form. \nOr tell if you want to browse more or search other item.")
      if user_intention == 'browse_more':
        amazon_service1(page='next', again=0)
      if user_intention == 'search_again':
        amazon_service1(page=0, again=1)

##### **Function 3 of Amazon Service**: 
1. Checks if the question asked by user is a product specific question or asking to do sentiment analysis of the customes who bought the product.
2. If it is a product specific question, then reviews are used as context and using question is answered using the question-answer model from that context.
3. If the question asked by user is asking to do sentiment analysis, then sentiment analysis model will be run and setiments in percentage will be printed out.

In [179]:
def amazon_service3():
  user_question = input()
  input_embedding = embedding_model.encode([user_question])
  service_corpus = {'question_intention':['Tell me about the product', 'Is this product durable', 'Which material is it made of', 'Will it serve the purpose', \
                                          'Is there other variants of this product'],\
            'sentiment_intention': ['Ok do the sentiment analysis', 'How many are happy or sad about the product', 'I want to know how many are happy and \
            how many are not', 'I want to know sentiment']}
  
  # Checking with which type of sentencem user's sentence is having max cosine similarity
  user_intention_list = [np.max([cosine(input_embedding, i) for i in embeddings]) for embeddings in [embedding_model.encode(types) for types in service_corpus.values()]]
  user_intention = list(service_corpus.keys())[np.argmax(user_intention_list)]
  if user_intention == 'question_intention':
    mprint("I'll answer that but if the reviews are very big, it will take some time. Average half an hour :|")
    answer_the_question(question=user_question)
  if user_intention == 'sentiment_intention':
    mprint("Ok hold on. I'll come up with the answer in percentage")
    link_to_product_page = product_r[current_page][3][user_product_number_choice]
    all_reviews = get_reviews_for_product(link_to_product_page)
    mprint('Reviews collected. Now lets do the sentiment analysis')
    amazon_review_sentiment(all_reviews)
    mprint('Select a different product number now')
    amazon_service2()

##### **Question answering part of Amazon Service**: 
1. Collects the product reviews data. If he will ask the question for a second time, then previously saved review data will be used.
2. Answer is made taking context from each of the reviews, whichever answer has highest score will be chosen.
3. Two of the highest scoring answers are chosen and returned from this function.
4. In the end, if user want to ask more question then the current function will be called again in itself (recursively) otherwise chatbot ends.

In [ ]:
def answer_the_question(question, more=0):
  if more == 0:
    # Get the user reviews of the selected product
    link_to_product_page = product_r[current_page][3][user_product_number_choice]
    global all_reviews
    all_reviews = get_reviews_for_product(link_to_product_page)
    mprint('Reviews collected. Now lets answer your question')
  if len(all_reviews) == 0:
    mprint('No reviews are available for this product. Choose different number')
    amazon_service2()
  else:
    scores = []
    answers = []
    for review in all_reviews:
      if len(review) < 25:
        continue
      # Get the answer
      QA_input = {'question': question, \
        'context': review}
      result = qa_model(QA_input)
      scores.append(result['score'])
      answers.append(result['answer'])
    best_loc = np.argsort(scores)[-2:]
    best_scores = np.array(scores)[best_loc]
    best_reviews = np.array(all_reviews)[best_loc]
    best_answers = np.array(answers)[best_loc]

    # #Print the answer:
    mprint('Top 2 Answers')
    mprint(f'1. {best_answers[0]} ({round(100*best_scores[0], 2)}%)\n2. {best_answers[1]} ({round(best_scores[1]*100, 2)}%)')
    print('\n\n')
    mprint('If you want to ask more, say exactly "more"')
    user_more = input()
    if user_more == "more":
      mprint('Type in your question')
      new_question = input()
      answer_the_question(question=new_question, more=1)
    else:
      mprint('Thank you for your time.')

In [ ]:
def twitter_service():
  mprint("####  LET'S BEGIN  #####")
  mprint("\n Write down a keyword i.e. A person's name, an Event, a news, etc.")
  user_input = input()
  mprint('\nOk, now I guess you will agree to give me some time for analysing your request.\n')

In [ ]:
def begin_chatbot():
  # Begin level 1
  greetings()
  about_service()

## Experimental Ground

In [ ]:
## Open and close a browser with selenium
#start the browser
driver = webdriver.Chrome(options=options)
#get the current title
driver.title

## BEGIN THE CHATBOT
begin_chatbot()

Output hidden; open in https://colab.research.google.com to view.